In [ ]:
!pip install streamlit

In [3]:
import os

os.chdir('/content/drive/MyDrive/Omdena/Osun Nigeria/OOP_App')

!pwd

/content/drive/MyDrive/Omdena/Osun Nigeria/OOP_App


In [8]:
%%writefile app.py

import streamlit as st
from apps import home, crop_disease_detector, crop_detector
from PIL import Image

with st.container():
    proj_title_col, logo_col = st.columns([6,1])

    with proj_title_col:
      st.subheader('Improving Food Security in Nigeria')

    with logo_col:
      logo = Image.open('/content/drive/MyDrive/Omdena/Osun Nigeria/App/osun_chapter.png')
      logo = logo.resize((75,75))
      st.image(logo)

PAGES = {
    "Home": home,
    "Crop Disease Detection": crop_disease_detector,
    'Crop Classification': crop_detector
}

selection = st.sidebar.radio("Menu", list(PAGES.keys()))
page = PAGES[selection]

page.app()

Overwriting app.py


In [ ]:
%%writefile apps/home.py

import streamlit as st

def app():
  st.subheader('Problem Statement')
  st.write("According to the Food and Agriculture Organisation of the United Nations, 2018, \
  approximately 88 % of the farmers in Nigeria engage in \
  agricultural production at a subsistence level, and they lack sustainable \
  farming knowledge and practices. Also, Nigeria is endowed with \
  different climatic conditions and soil quality which leads to lackluster \
  crop production. ")

  st.markdown('##')

  st.subheader('Objectives of the Project')
  st.write('This project is aimed at helping farmers to boost their \
  farm produce and plan their farming system.')

Writing apps/home.py


In [29]:
%%writefile apps/app_detector.py

import streamlit as st
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import img_to_array
from PIL import Image
import cv2
import utils

class AppDetector:

  def __init__(self, data, target_size):
    self.data = data
    self.target_size = target_size
  

  def app(self):
    new_image = ''
    has_pred = False
    has_replaced_image = False
    pred = ''

    model = utils.load_model(self.data)

    
    with st.container():
        upload_col, predict_col = st.columns([6,1])
        with upload_col:
          
          #Upload the image
          uploaded_file = st.file_uploader('Choose a file',type=["jpg", 'png'])
          if uploaded_file is not None:
            img = Image.open(uploaded_file)
            img_display = img.resize((700, 500))
            st.image(img_display)

            #Preprocess image
            if self.data == 'Crops':
              img = utils.crop_preprocess_img(img, self.target_size)
            else:
              img = utils.disease_preprocess_img(img, self.target_size)
            
            new_image = img
            has_replaced_image = True
            
            
        with predict_col:
          #Predict the disease label for the image
            if st.button('Predict'):
              if has_replaced_image:
                predictions = utils.predict(new_image, model)
                pred = np.argmax(predictions)
                has_pred = True
              else:
                st.error('Please upload an image')

    #This container is for the description of the predicted disease of the image            
    with st.container():
      if has_pred:
        if self.data == 'Crops':
          st.title(utils.get_class(self.data, pred))
          st.write(predictions)
        else:
          with st.expander("Read disease description"):
            disease = utils.get_class(self.data, pred)
            st.title(disease)
            disease_desc = utils.get_description(disease)
            st.markdown(disease_desc, unsafe_allow_html=True)

Overwriting apps/app_detector.py


In [ ]:
%%writefile apps/crop_disease_detector.py

import streamlit as st
from apps.app_detector import AppDetector

TARGET_SIZE = 100

def app(): 
  selection = st.selectbox('Crop Dataset', ('Maize', 'Rice'))
  app_detect = AppDetector(selection, TARGET_SIZE)
  app_detect.app()

Overwriting apps/crop_disease_detector.py


In [ ]:
%%writefile apps/crop_detector.py

import streamlit as st
from apps.app_detector import AppDetector

TARGET_SIZE = 224

def app():
  app_detect = AppDetector('Crops', TARGET_SIZE)
  app_detect.app()

Writing apps/crop_detector.py


In [32]:
%%writefile utils.py

import streamlit as st
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import img_to_array
from PIL import Image
import cv2


def load_model(dataset):
  """
  Loads the models.
  """

  if dataset == 'Maize':
    model = tf.keras.models.load_model('/content/drive/MyDrive/Omdena/Osun Nigeria/Model/model_maize_final.h5')
  elif dataset == 'Rice':
    model = tf.keras.models.load_model('/content/drive/MyDrive/Omdena/Osun Nigeria/Model/model_rice.h5')
  elif dataset == 'Crops':
    model = tf.keras.models.load_model('/content/drive/MyDrive/crop_classification.h5')
  return model

def crop_preprocess_img(image, TARGET_SIZE):
  pil_img_rgb = image.convert('RGB') 
  cv2_img = np.array(pil_img_rgb) #convert PIL Image to cv2 format
  #img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #convert BGR to RGB
  img = cv2.resize(cv2_img, (TARGET_SIZE, TARGET_SIZE)) #resize the image to (224, 224)
  img = img_to_array(img) #convert to numpy array
  img = img/255 #normalize the image
  img = np.expand_dims(img, axis = 0) #expands dimension to one sample
  return img

def disease_preprocess_img(image, TARGET_SIZE):
  """ Preprocess the image in the same way the images for model building were processed. 
  
  image: uploaded image

  Returns: image preprocessed using resnet50's preprocess_input
  """

  img = image.resize((TARGET_SIZE, TARGET_SIZE))
  img = img_to_array(img) #convert to numpy array
  img = np.expand_dims(img, axis = 0) #expands dimension to one sample
  img = preprocess_input(img) #preprocess function by resnet50
  return img

def predict(image, model):
  """ Predicts the disease of the image.

  image: preprocessed image
  model: model for maize or rice leaf disease datasets

  Returns: probability for each class/disease
  """
  predictions = model.predict(image)
  return predictions

def get_class(crop, idx):
  """ Gets the disease label for the predicted image.

  crop: crop type - maize or rice
  idx: column index of the class with the highest probability in the predictions

  Returns: disease label

  """
  if crop == 'Maize':
    CLASSES = {
        0: 'Blight',
        1: 'Common Rust',
        2: 'Gray Leaf Spot',
        3: 'Healthy'
    }
  elif crop == 'Rice':
    CLASSES = {
        0: 'Bacterial Leaf Blight',
        1: 'Brown Spot',
        2: 'Leaf Smut'
    }
  elif crop == 'Crops':
    CLASSES = {
        0: 'Rice',
        1: 'Maize'
    }

  return CLASSES[idx]

def get_description(disease):
  """ Gets the disease description

    disease: predicted disease for the image
  """
  maize_blight = f"""
  <p> 
  The tan lesions of northern corn leaf blight are slender and oblong tapering at the ends ranging in size between 1 to 6 inches. </p>
  <p>Lesions run parallel to the leaf margins beginning on the lower leaves and moving up the plant. They may coalesce and cover the enter leaf.
  </p>
  <p>Spores are produced on the underside of the leaf below the lesions giving the appearance of a dusty green fuzz.</p>

  <p> <i>Source: https://cals.cornell.edu/field-crops/corn/diseases-corn/</i></p>
  """

  maize_common_rust = f"""
  <p> Common rust is caused by the fungus Puccinia sorghi. </p>
  <p>Small, round to elongate brown pustules form on both leaf surfaces and other above ground parts of the plant. </p>
  <p>As the pustules mature they become brown to black. If disease is severe, the leaves may yellow and die early
  </p>

  <p> <i>Source: https://cals.cornell.edu/field-crops/corn/diseases-corn/</i></p>
  """
  maize_gray_leaf_spot = f"""
  <p> Gray leaf spot is caused by the fungus Cercospora zeae-maydis. 
  Lesions start as a small dot surrounded by yellow halo, and then will elongate over time parallel to the veins becoming pale brown to gray.
  </p>

  <p> <i>Source: https://cals.cornell.edu/field-crops/corn/diseases-corn/</i></p>
  """

  maize_healthy = f"""
  <p> Healthy
  </p>
  """

  rice_bacterial_leaf_blight = f"""
  <p> Rice Bacterial Blight is a deadly bacterial disease that is among the most destructive afflictions of cultivated rice (Oryza sativa and O. glaberrima). 
  In severe epidemics, crop loss may be as high as 75 percent, and millions of hectares of rice are infected annually.
  </p>
  """

  rice_brown_spot = f"""
  <p> Brown spot is caused by the fungus Cochliobolus miyabeanus. Also called Helminthosporium leaf spot, it is one of the most prevalent rice diseases. 
  It is a fungal disease that infects the coleoptile, leaves, leaf sheath, panicle branches, glumes, and spikelets.
  </p>
  """

  rice_leaf_smut = f"""
  <p> Leaf smut, caused by the fungus Entyloma oryzae, is a widely distributed, but somewhat minor, disease of rice. 
  The fungus produces slightly raised, angular, black spots (sori) on both sides of the leaves.
  </p>
  """

  dict_description = {
      'Blight': maize_blight,
      'Common Rust': maize_common_rust,
      'Gray Leaf Spot': maize_gray_leaf_spot,
      'Healthy': maize_healthy,
      'Bacterial Leaf Blight': rice_bacterial_leaf_blight,
      'Brown Spot': rice_brown_spot,
      'Leaf Smut': rice_leaf_smut
  }

  return dict_description[disease]


Overwriting utils.py


In [33]:
!streamlit run app.py & npx localtunnel --port 8501

npx: installed 22 in 2.785s
your url is: https://white-bullfrog-36.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.82.228.228:8501

2021-10-29 17:39:33.732947: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-29 17:39:33.733031: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (b81b54e114ac): /proc/driver/nvidia/version does not exist
2021-10-29 17:39:51.292838: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-29 17:54:18.615 5 out of the last 5 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7ff8262c0f80> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.functi